In [2]:
import os
import shutil
import xml.etree.ElementTree as ET

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [6]:
# Пути для сохранения разделенных выборок
images_train_dir = "../../data/processed/for YOLO detector/train/images"
labels_train_dir = "../../data/processed/for YOLO detector/train/labels"

images_valid_dir = "../../data/processed/for YOLO detector/valid/images"
labels_valid_dir = "../../data/processed/for YOLO detector/valid/labels"

images_test_dir = "../../data/processed/for YOLO detector/test/images"
labels_test_dir = "../../data/processed/for YOLO detector/test/labels"

# Создание каталогов для train, valid, test и images
os.makedirs(images_train_dir, exist_ok=True)
os.makedirs(labels_train_dir, exist_ok=True)

os.makedirs(images_valid_dir, exist_ok=True)
os.makedirs(labels_valid_dir, exist_ok=True)

os.makedirs(images_test_dir, exist_ok=True)
os.makedirs(labels_test_dir, exist_ok=True)


# Список для хранения путей к изображениям и разметкам
images = []
annotations = []

###############################################################################
############# Формируем датасет из каталога Губернаторские отчеты #############
###############################################################################

# Путь к каталогу с данными
data_dir = "../../data/raw/Распознавание текстов/Губернаторские отчеты"

# Перебор всех каталогов и файлов внутри data_dir
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".JPG"):
            image_path = os.path.join(root, file)
            annotation_path = os.path.join(root, file.replace(".JPG", "_pvoc_imglab.xml"))
            if os.path.exists(annotation_path):
                images.append(image_path)
                annotations.append(annotation_path)

########################################################################################
############# Формируем датасет из каталога Уставные грамоты – Афанасенков #############
########################################################################################

# Путь к каталогу с данными
data_dir = "../../data/raw/Распознавание текстов/Уставные грамоты – Афанасенков"
# Перебор всех каталогов и файлов внутри data_dir
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            annotation_path = os.path.join(root, file.replace(".jpg", "_pvoc_imglab.xml"))
            if os.path.exists(annotation_path):
                images.append(image_path)
                annotations.append(annotation_path)

############################################################################################
############# Формируем датасет из каталога Уставные грамоты в jpg (Просветов) #############
############################################################################################

image_dir = "../../data/raw/Распознавание текстов/Уставные грамоты в jpg (Просветов)"
annotation_dir = "../../data/raw/Распознавание текстов/Уставные грамоты в jpg (Просветов)/Обработка/Просветов (13.12)"

# Перебор всех файлов изображений в image_dir
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            annotation_file = file.replace(".jpg", "_pvoc_imglab.xml")
            annotation_path = os.path.join(annotation_dir, annotation_file)
            if os.path.exists(annotation_path):
                images.append(image_path)
                annotations.append(annotation_path)


# Разделение выборки на train, valid и test
train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.2, random_state=42)
train_images, valid_images, train_annotations, valid_annotations = train_test_split(train_images, train_annotations, test_size=0.25, random_state=42)

# Функция для копирования файлов
def copy_files(images, annotations, dest_dir):
    for image, annotation in zip(images, annotations):
        shutil.copy(image, dest_dir)
        shutil.copy(annotation, dest_dir)

# Функция для преобразования координат в относительные координаты
def convert_coordinates(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2]) / 2.0
    y = (box[1] + box[3]) / 2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    x_rel = x * dw
    w_rel = w * dw
    y_rel = y * dh
    h_rel = h * dh
    return x_rel, y_rel, w_rel, h_rel

# Функция для записи разметки в формате YOLO
def write_yolo_annotation(annotation_path, size, objects, output_file):
    with open(output_file, "w") as f:
        for obj in objects:
            x_rel, y_rel, w_rel, h_rel = convert_coordinates(size, obj["bbox"])
            # первый 0 - единственный класс разметки 'text'
            f.write(f"0 {x_rel} {y_rel} {w_rel} {h_rel}\n")

# Преобразование разметки для train выборки
def process_data(images, annotations, images_dir, labels_dir, data_type):
    
    for image_path, annotation_path in tqdm(zip(images, annotations), total=len(images), desc=f"Подготавливаем {data_type} выборку"):
        # Копирование изображений в директорию images
        image_name = os.path.basename(image_path)

        shutil.copy(image_path, images_dir + "/" + image_name)

        # Преобразование разметки в формат YOLO
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        txt_filename = image_name.replace(".JPG", ".txt")
        txt_path = labels_dir + "/" + txt_filename

        size = (int(root.find("size/width").text), int(root.find("size/height").text))
        objects = []
        for obj in root.findall("object"):
            name = obj.find("name").text
            bbox = [
                float(obj.find("bndbox/xmin").text),
                float(obj.find("bndbox/ymin").text),
                float(obj.find("bndbox/xmax").text),
                float(obj.find("bndbox/ymax").text),
            ]
            objects.append({"name": name, "bbox": bbox})

        write_yolo_annotation(annotation_path, size, objects, txt_path)

# Обработка данных для каждой выборки
process_data(train_images, train_annotations, images_train_dir, labels_train_dir, "обучающую")
process_data(valid_images, valid_annotations, images_valid_dir, labels_valid_dir, "валидационную")
process_data(test_images, test_annotations, images_test_dir, labels_test_dir, "тестовую")

248 248
